In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# !/usr/bin/env python
# coding: utf-8


# In[31]:


import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI, TUDataset
from torch_geometric.data import DataLoader

# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--base_model', type=str, default='GCN', help='propagation model for encoder',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--if_smoothed', action='store_true', default=False)
parser.add_argument('--encoder_model', type=str, default='Grace', help='propagation model for encoder',
                    choices=['Grace','GraphCL'])
parser.add_argument('--dataset', type=str, default='PROTEINS', 
                    help='Dataset',
                    choices=['PROTEINS','MUTAG'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units of GConv model.')
parser.add_argument('--num_hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=32,
                    help='Number of hidden units in MLP.')
# parser.add_argument('--thrd', type=float, default=0.5)
# parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=2,
                    help="Threshold of prunning edges")
# Contrastive Learning setting
parser.add_argument('--config', type=str, default="config.yaml")
parser.add_argument('--cl_lr', type=float, default=0.01)
# parser.add_argument('--cl_num_proj_hidden', type=int, default=128)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=1)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0.1)
parser.add_argument('--drop_edge_rate_2', type=float, default=0.1)
parser.add_argument('--drop_feat_rate_1', type=float, default=0.1)
parser.add_argument('--drop_feat_rate_2', type=float, default=0.1)
parser.add_argument('--drop_node_rate_1', type=float, default=0.1)
parser.add_argument('--drop_node_rate_2', type=float, default=0.1)
parser.add_argument('--tau', type=float, default=0.2)
parser.add_argument('--cl_num_epochs', type=int, default=100)
parser.add_argument('--cl_weight_decay', type=float, default=1e-5)
parser.add_argument('--batch_size', default=128, type=int,
                    help="batch_size of graph dataset")
parser.add_argument('--walk_length', default=10, type=int)
# parser.add_argument('--select_thrh', type=float, default=0.8)

# Attack
parser.add_argument('--attack', type=str, default='none',
                    choices=['nettack','random','none'],)
parser.add_argument('--select_target_ratio', type=float, default=0.1,
                    help="The number of selected target test nodes for targeted attack")
# Randomized Smoothing
parser.add_argument('--prob', default=0.8, type=float,
                    help="probability to keep the status for each binary entry")
# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)


# In[13]:


from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])


if(args.dataset == 'PROTEINS' or args.dataset == 'MUTAG'):
    dataset = TUDataset(root='./data/', name=args.dataset, transform=None,use_node_attr = True)

# data = dataset[0].to(device)
dataloader = DataLoader(dataset, batch_size=args.batch_size)


/tmp/ipykernel_1090869/817794642.py:14: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Namespace(add_edge_rate_1=0, add_edge_rate_2=0, attack='none', base_model='GCN', batch_size=128, cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.01, cl_num_epochs=100, cl_num_layers=2, cl_weight_decay=1e-05, config='config.yaml', cont_weight=1, cuda=True, dataset='PROTEINS', debug=True, device_id=2, drop_edge_rate_1=0.1, drop_edge_rate_2=0.1, drop_feat_rate_1=0.1, drop_feat_rate_2=0.1, drop_node_rate_1=0.1, drop_node_rate_2=0.1, dropout=0.5, encoder_model='Grace', hidden=32, if_smoothed=False, no_cuda=False, num_hidden=32, num_proj_hidden=32, prob=0.8, seed=10, select_target_ratio=0.1, tau=0.2, test_model='GCN', train_lr=0.01, walk_length=10, weight_decay=0.0005)


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [15]:
from torch_geometric.utils import mask_feature,add_random_edge,dropout_adj
from torch_geometric.utils import to_undirected, to_dense_adj,to_torch_coo_tensor,dense_to_sparse
from torch.distributions.bernoulli import Bernoulli
from GCL.eval import get_split

for data in dataset:
    data.edge_index = to_undirected(data.edge_index)
    
# for data in dataset:
#     print(5/data.edge_index.shape[1])
#     print(data.edge_index.shape,)
#     edge_index, added_edges = add_random_edge(data.edge_index,p=5/data.edge_index.shape[1],force_undirected=False,)
#     # edge_index = to_undirected(edge_index)
#     row = torch.cat([added_edges[0], added_edges[1]])
#     col = torch.cat([added_edges[1],added_edges[0]])
#     added_edges = torch.stack([row,col])
#     edge_index = torch.cat([data.edge_index,added_edges],dim=1)
#     print(edge_index.shape)
#     break

# def sample_noise_all_dense(args,edge_index,edge_weight,device):
#     adj = to_dense_adj(edge_index,edge_attr=edge_weight)[0]
#     row_idx, col_idx = np.triu_indices(adj.shape[1])
#     rand_inputs = torch.randint_like(adj[row_idx,col_idx], low=0, high=2, device=device)
#     adj_noise = torch.zeros(adj.shape, device=device)
#     m = Bernoulli(torch.tensor([args.prob]).to(device))
#     mask = m.sample(adj[row_idx,col_idx].shape).squeeze(-1).int()
#     adj_noise[row_idx,col_idx] = adj[row_idx,col_idx] * mask + rand_inputs * (1 - mask)
#     adj_noise = adj_noise + adj_noise.t()
#     ## diagonal elements set to be 0
#     ind = np.diag_indices(adj_noise.shape[0]) 
#     adj_noise[ind[0],ind[1]] = adj[ind[0], ind[1]]
#     edge_index, edge_weight = dense_to_sparse(adj_noise)
#     return edge_index,edge_weight

# def sample_noise_1by1_dense(args,edge_index,edge_weight,idx,device):
#     adj = to_dense_adj(edge_index,edge_attr=edge_weight)[0]
#     adj_noise = adj.clone().detach()
#     # row_idx, col_idx = np.triu_indices(adj.shape[1])
#     rand_inputs = torch.randint_like(adj[idx], low=0, high=2, device=device)
#     # adj_noise = torch.zeros(adj.shape, device=device)
#     m = Bernoulli(torch.tensor([args.prob]).to(device))
#     mask = m.sample(adj[idx].shape).squeeze(-1).int()
#     adj_noise[idx] = adj[idx] * mask + rand_inputs * (1 - mask)
#     adj_noise[idx,idx] = adj[idx,idx]
#     print(adj_noise)
#     adj_noise[:,idx] = adj_noise[idx].t()
#     # adj_noise = adj_noise + adj_noise.t()
#     # ## diagonal elements set to be 0
#     # ind = np.diag_indices(adj_noise.shape[0]) 
#     # adj_noise[ind[0],ind[1]] = adj[ind[0], ind[1]]
#     edge_index, edge_weight = dense_to_sparse(adj_noise)
#     return edge_index,edge_weight


# for data in dataset:
#     data = data.to(device)
#     split = get_split(num_samples=data.x.shape[0], train_ratio=0.8, test_ratio=0.1)
#     sample_noise_1by1_dense(args,data.edge_index,data.edge_weight,split['test'],device)
#     # sparse_adj = to_torch_coo_tensor(data.edge_index,data.edge_weight)
#     # print(sparse_adj.shape)
#     # print(adj_noise.nonzero())
#     # rand_inputs
#     break
for data in dataloader:
    print(data.)
    break

tensor([  0,   0,   0,  ..., 127, 127, 127])


In [ ]:
# if(args.dataset == 'ogbn-arxiv'):
#     nNode = data.x.shape[0]
#     setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
#     # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
#     data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
#     data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
#     data.y = data.y.squeeze(1)
# # we build our own train test split 
# print(data)

# In[14]:


# # from utils import get_split
# # # data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)
# idx_train = data.train_mask.nonzero().flatten()
# idx_val = data.val_mask.nonzero().flatten()
# idx_clean_test = data.test_mask.nonzero().flatten()

# from torch_geometric.utils import to_undirected
# from utils import subgraph
# data.edge_index = to_undirected(data.edge_index)
# train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
# mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# # filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
# unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()


# In[28]:


if(args.dataset == 'Cora'):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.4
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.4
    args.tau = 0.1
    args.cl_lr = 0.0005
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 128
    args.hidden = 128
    args.num_proj_hidden = 128
elif(args.dataset == "Pubmed"):
    args.drop_edge_rate_1 = 0.4
    args.drop_edge_rate_2 = 0.1
    args.drop_feat_rate_1 = 0.0
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256
    args.hidden = 256
elif(args.dataset == "Citeseer"):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.0
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256
    args.hidden = 256

# In[29]:
import copy 
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Grace

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation,label_classification_origin

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

import GCL.augmentors as A
from models.random_smooth import sample_noise,sample_noise_1by1,sample_noise_all
from models.GraphCL import GConv, Encoder


data = data.to(device)
num_class = int(data.y.max()+1)
input_dim = max(dataset.num_features, 1)

rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)

accs = []
for seed in seeds:
    # Construct and train encoder
    aug1 = A.Identity()
    aug2 = A.RandomChoice([A.RWSampling(num_seeds=args.seed, walk_length=args.walk_length),
                           A.NodeDropping(pn=args.drop_edge_rate_2),
                           A.FeatureMasking(pf=args.drop_feat_rate_2),
                           A.EdgeRemoving(pe=args.drop_node_rate_2)], 1)

    gconv = GConv(input_dim=input_dim, hidden_dim=args.hidden, num_layers=2).to(device)
    model = Encoder(args = args, encoder=gconv, augmentor=(aug1, aug2), input_dim=input_dim, hidden_dim=args.num_hidden, lr=args.cl_lr, tau=args.tau,num_epoch = args.cl_num_epochs, if_smoothed = args.if_smoothed,device = device)
    model.fit(dataloader)
    test_result = model.test(dataloader)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
    
# In[30]:
    from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
    from torch_geometric.utils import from_scipy_sparse_matrix
    from scipy.sparse import csr_matrix
    def single_test(adj, features, target_node, gcn=None):
        if gcn is None:
            # test on GCN (poisoning attack)
            gcn = GCN(nfeat=features.shape[1],
                    nhid=16,
                    nclass=labels.max().item() + 1,
                    dropout=0.5, device=device)

            gcn = gcn.to(device)

            gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
            gcn.eval()
            output = gcn.predict()
        else:
            # test on GCN (evasion attack)
            output = gcn.predict(features, adj)
        probs = torch.exp(output[[target_node]])

        # acc_test = accuracy(output[[target_node]], labels[target_node])
        acc_test = (output.argmax(1)[target_node] == labels[target_node])
        return acc_test.item()

    # Evaluation Metric: Robust Accuracy
    # Node-level task
    print(args.attack)
    if(args.attack == 'random'):
        import construct_graph
        import copy
        perturbation_sizes = list(range(1,6))
        for n_perturbation in perturbation_sizes:
            print("Perturbation Size:{}".format(n_perturbation))
            noisy_data = copy.deepcopy(data)
            print(n_perturbation)
            for idx in idx_clean_test:
                noisy_data = construct_graph.generate_node_noisy(args,noisy_data,idx,n_perturbation,device)
                noisy_data = noisy_data.to(device)
            model.eval()
            if(args.if_smoothed):
                # noisy_data.edge_index,noisy_data.edge_weight = sample_noise_1by1(args, noisy_data.x, noisy_data.edge_index, noisy_data.edge_weight,idx_clean_test,device)
                noisy_data.edge_index,noisy_data.edge_weight = sample_noise(args,noisy_data.edge_index, noisy_data.edge_weight,idx_clean_test, device)
                z, _, _ = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)
            else:
                z, _, _ = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)
            acc = label_evaluation(z, noisy_data.y, idx_train, idx_clean_test)
            print("Accuracy:",acc)
    elif(args.attack == 'none'):
        model.eval()
        if(args.if_smoothed == True):
            rs_edge_index, rs_edge_weight = sample_noise_all(args, data.edge_index, data.edge_weight, device)
            z, _, _ = model(data.x, rs_edge_index, rs_edge_weight)
        else:
            z, _, _ = model(data.x, data.edge_index, data.edge_weight)
            # acc = label_evaluation(z, data.y, idx_train, idx_clean_test)
            # print("Accuracy:",acc)

In [18]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# !/usr/bin/env python
# coding: utf-8


# In[31]:


import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI, TUDataset
from torch_geometric.loader import DataLoader

# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--base_model', type=str, default='GCN', help='propagation model for encoder',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--if_smoothed', action='store_true', default=False)
parser.add_argument('--encoder_model', type=str, default='Grace', help='propagation model for encoder',
                    choices=['Grace','GraphCL'])
parser.add_argument('--dataset', type=str, default='PROTEINS', 
                    help='Dataset',
                    choices=['PROTEINS','MUTAG'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units of GConv model.')
parser.add_argument('--num_hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=32,
                    help='Number of hidden units in MLP.')
# parser.add_argument('--thrd', type=float, default=0.5)
# parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=2,
                    help="Threshold of prunning edges")
# Contrastive Learning setting
parser.add_argument('--config', type=str, default="config.yaml")
parser.add_argument('--cl_lr', type=float, default=0.01)
# parser.add_argument('--cl_num_proj_hidden', type=int, default=128)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=1)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0.1)
parser.add_argument('--drop_edge_rate_2', type=float, default=0.1)
parser.add_argument('--drop_feat_rate_1', type=float, default=0.1)
parser.add_argument('--drop_feat_rate_2', type=float, default=0.1)
parser.add_argument('--drop_node_rate_1', type=float, default=0.1)
parser.add_argument('--drop_node_rate_2', type=float, default=0.1)
parser.add_argument('--tau', type=float, default=0.2)
parser.add_argument('--cl_num_epochs', type=int, default=100)
parser.add_argument('--cl_weight_decay', type=float, default=1e-5)
parser.add_argument('--batch_size', default=128, type=int,
                    help="batch_size of graph dataset")
parser.add_argument('--walk_length', default=10, type=int)
# parser.add_argument('--select_thrh', type=float, default=0.8)

# Attack
parser.add_argument('--attack', type=str, default='none',
                    choices=['nettack','random','none'],)
parser.add_argument('--select_target_ratio', type=float, default=0.1,
                    help="The number of selected target test nodes for targeted attack")
# Randomized Smoothing
parser.add_argument('--prob', default=1.0, type=float,
                    help="probability to keep the status for each binary entry")
# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)


# In[13]:


from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])


if(args.dataset == 'PROTEINS' or args.dataset == 'MUTAG'):
    dataset = TUDataset(root='./data/', name=args.dataset, transform=None,use_node_attr = True)

data = dataset[0].to(device)

from torch_geometric.utils import to_undirected
for data in dataset:
    data.edge_index = to_undirected(data.edge_index)
dataloader = DataLoader(dataset, batch_size=args.batch_size)

# if(args.dataset == 'ogbn-arxiv'):
#     nNode = data.x.shape[0]
#     setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
#     # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
#     data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
#     data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
#     data.y = data.y.squeeze(1)
# # we build our own train test split 
# print(data)

# In[14]:


# # from utils import get_split
# # # data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)
# idx_train = data.train_mask.nonzero().flatten()
# idx_val = data.val_mask.nonzero().flatten()
# idx_clean_test = data.test_mask.nonzero().flatten()

# from torch_geometric.utils import to_undirected
# from utils import subgraph
# data.edge_index = to_undirected(data.edge_index)
# train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
# mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# # filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
# unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()


# In[28]:


if(args.dataset == 'Cora'):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.4
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.4
    args.tau = 0.1
    args.cl_lr = 0.0005
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 128
    args.hidden = 128
    args.num_proj_hidden = 128
elif(args.dataset == "Pubmed"):
    args.drop_edge_rate_1 = 0.4
    args.drop_edge_rate_2 = 0.1
    args.drop_feat_rate_1 = 0.0
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256
    args.hidden = 256
elif(args.dataset == "Citeseer"):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.0
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256
    args.hidden = 256

# In[29]:


import copy 
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Grace

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation,label_classification_origin

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

import GCL.augmentors as A
from models.random_smooth import sample_noise,sample_noise_1by1,sample_noise_all
from models.GraphCL import GConv, Encoder


data = data.to(device)
num_class = int(data.y.max()+1)
input_dim = max(dataset.num_features, 1)

rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)

accs = []
for seed in seeds:
    # Construct and train encoder
    aug1 = A.Identity()
    aug2 = A.RandomChoice([A.RWSampling(num_seeds=args.seed, walk_length=args.walk_length),
                           A.NodeDropping(pn=args.drop_edge_rate_2),
                           A.FeatureMasking(pf=args.drop_feat_rate_2),
                           A.EdgeRemoving(pe=args.drop_node_rate_2)], 1)

    gconv = GConv(input_dim=input_dim, hidden_dim=args.hidden, num_layers=2).to(device)
    model = Encoder(args = args, encoder=gconv, augmentor=(aug1, aug2), input_dim=input_dim, hidden_dim=args.num_hidden, lr=args.cl_lr, tau=args.tau,num_epoch = args.cl_num_epochs, if_smoothed = args.if_smoothed,device = device)
    model.fit(dataloader)
    test_result = model.test(dataloader)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
    
# In[30]:
    # Evaluation Metric: Robust Accuracy
    # Node-level task
    print(args.attack)
    if(args.attack == 'random'):
        import construct_graph
        import copy
        perturbation_sizes = list(range(1,6))
        for n_perturbation in perturbation_sizes:
            print("Perturbation Size:{}".format(n_perturbation))
            # noisy_dataset = copy.deepcopy(dataset)
            print(n_perturbation)
            noisy_dataset = []
            for data in dataset:
            # for i in range(len(noisy_dataset)):
                noisy_data = construct_graph.generate_graph_noisy(args,data,n_perturbation,device)
                # noisy_dataset[i] = noisy_data.to(device)
                noisy_dataset.append(noisy_data)
            noisy_dataloader = DataLoader(noisy_dataset, batch_size=args.batch_size)
            # test_result = model.single_test(noisy_dataloader)
            # for i in range(len(noisy_dataset)):
            #     noisy_data = construct_graph.generate_graph_noisy(args,noisy_dataset[i],n_perturbation,device)
            #     # noisy_dataset[i] = noisy_data.to(device)
            #     # noisy_dataloader = DataLoader(noisy_dataset, batch_size=args.batch_size)
            #     test_result = model.single_test(noisy_data)
            print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
            
    # elif(args.attack == 'none'):
    #     model.eval()
    #     if(args.if_smoothed == True):
    #         rs_edge_index, rs_edge_weight = sample_noise_all(args, data.edge_index, data.edge_weight, device)
    #         z, _, _ = model(data.x, rs_edge_index, rs_edge_weight)
    #     else:
    #         z, _, _ = model(data.x, data.edge_index, data.edge_weight)
        # acc = label_evaluation(z, data.y, idx_train, idx_clean_test)
        # print("Accuracy:",acc)


# In[ ]:






Namespace(add_edge_rate_1=0, add_edge_rate_2=0, attack='none', base_model='GCN', batch_size=128, cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.01, cl_num_epochs=100, cl_num_layers=2, cl_weight_decay=1e-05, config='config.yaml', cont_weight=1, cuda=True, dataset='PROTEINS', debug=True, device_id=2, drop_edge_rate_1=0.1, drop_edge_rate_2=0.1, drop_feat_rate_1=0.1, drop_feat_rate_2=0.1, drop_node_rate_1=0.1, drop_node_rate_2=0.1, dropout=0.5, encoder_model='Grace', hidden=32, if_smoothed=False, no_cuda=False, num_hidden=32, num_proj_hidden=32, prob=1.0, seed=10, select_target_ratio=0.1, tau=0.2, test_model='GCN', train_lr=0.01, walk_length=10, weight_decay=0.0005)


(T): 100%|██████████| 100/100 [00:16<00:00,  5.97it/s, loss=15.7]
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed

(E): Best test F1Mi=0.7117, F1Ma=0.7077
none


/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mfl5681/anaconda3/envs/py38_torch120/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
import construct_graph
import copy
perturbation_sizes = list(range(1,6))
for n_perturbation in perturbation_sizes:
    print("Perturbation Size:{}".format(n_perturbation))
    # noisy_dataset = copy.deepcopy(dataset)
    print(n_perturbation)
    noisy_dataset = []
    for data in dataset:
    # for i in range(len(noisy_dataset)):
        noisy_data = construct_graph.generate_graph_noisy(args,data,n_perturbation,device)
        # noisy_dataset[i] = noisy_data.to(device)
        noisy_dataset.append(noisy_data)
    noisy_dataloader = DataLoader(noisy_dataset, batch_size=args.batch_size)
    # test_result = model.single_test(noisy_dataloader)
    # for i in range(len(noisy_dataset)):
    #     noisy_data = construct_graph.generate_graph_noisy(args,noisy_dataset[i],n_perturbation,device)
    #     # noisy_dataset[i] = noisy_data.to(device)
    #     # noisy_dataloader = DataLoader(noisy_dataset, batch_size=args.batch_size)
    #     test_result = model.single_test(noisy_data)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
    

Perturbation Size:1
1
(E): Best test F1Mi=0.7117, F1Ma=0.7077
Perturbation Size:2
2
(E): Best test F1Mi=0.7117, F1Ma=0.7077
Perturbation Size:3
3
(E): Best test F1Mi=0.7117, F1Ma=0.7077
Perturbation Size:4
4
(E): Best test F1Mi=0.7117, F1Ma=0.7077
Perturbation Size:5
5
(E): Best test F1Mi=0.7117, F1Ma=0.7077


In [22]:
for data in dataloader:
    print(data)
    break
for data in noisy_dataloader:
    print(data)
    break

DataBatch(edge_index=[2, 27320], x=[7265, 4], y=[128], batch=[7265], ptr=[129])
DataBatch(edge_index=[2, 28600], x=[7265, 4], y=[128], batch=[7265], ptr=[129])
